In [78]:
import os
import sys
import re
import string
import argparse
from numpy import random

try:
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    from nltk import pos_tag
except:
    print("NLTK NOT FOUND")

In [89]:
checkword = "hard"

In [98]:
tmp_dominant_only = 1

# datafile = "~/Data/one-million-sense-tagged-instances-wn30"
split = 0.4
window = 1
windowSize = 5
add_start_end = 1
dominant_only = 0
dominant_per = 0.2
remove_punctuation = 0
remove_stopwords = 0
POS_tags = 0
POS_RED = 0
senseval = True
pos="noun"

Full_Sent = open('Sentences.txt', 'r').read().split('\n')
Full_Sens = open('Senses.txt', 'r').read().split('\n')
print("Total No. of Senses: " + str(len(Full_Sens)))
print("Total No. of Sentences: " + str(len(Full_Sent)))
sentStr = ""
sensStr = ""

wordList = {
        'hard': ['HARD1', 'HARD2', 'HARD3'],
        'interest': ['interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'interest6'],
        'line': ['text', 'phone', 'product', 'formation', 'division', 'cord'],
        'serve': ['SERVE2', 'SERVE6', 'SERVE10', 'SERVE12']
        }

checklist = wordList[checkword]

for i in range(0, len(Full_Sens)):
    if Full_Sens[i] in checklist:
        sentStr += Full_Sent[i] + "\n"
        sensStr += Full_Sens[i] + "\n"

sentFile = open('Preprocess_Files/' + checkword + '/sent', 'w')
sensFile = open('Preprocess_Files/' + checkword + '/sense', 'w')

sentFile.write(sentStr)
sensFile.write(sensStr)

sentFile.close()
sensFile.close()

Sent = open('Preprocess_Files/' + checkword + '/sent', "r").read().split('\n')
del Sent[len(Sent) - 1] # last string is empty
Sens = open('Preprocess_Files/' + checkword + '/sense', "r").read().split()

assert(len(Sent) == len(Sens))
print("Using Senseval Dataset")
print("For Word: " + checkword)
print("Total number of examples: " + str(len(Sent)))
SenSet = list(set(Sens))

# Delete minor senses if flag positive
if dominant_only == 0:
    SenCnt = []
    for i in SenSet:
        SenCnt.append(Sens.count(i))

    maxCnt = max(SenCnt)
    SenRemove = []
    for i in range(0, len(SenSet)):
        if SenCnt[i] < dominant_per * maxCnt:
            print("IGNORING", SenSet[i], SenCnt[i])
            SenRemove.append(SenSet[i])

    tmpSt = []
    tmpSe = []
    for i in range(0, len(Sent)):
        if Sens[i] not in SenRemove:
            tmpSt.append(Sent[i])
            tmpSe.append(Sens[i])
    Sent = tmpSt
    Sens = tmpSe

Total No. of Senses: 15226
Total No. of Sentences: 15226
Using Senseval Dataset
For Word: hard
Total number of examples: 4333
IGNORING HARD2 502
IGNORING HARD3 376


In [100]:
Sent

["`` He may lose all popular support  ,   but someone has to kill him to defeat him and that  ' s  HARD  to do .   '  '  ",
 "Clever White House `` spin doctors  '  '  are having a  HARD  time helping President Bush explain away the economic bashing that low-and middle-income workers are taking these days  .  ",
 "I find it  HARD  to believe that the Sacramento River will ever be quite the same  ,   although I certainly wish that I ' m wrong  .  ",
 'Now when you get bad credit data or are confused with another person  ,   the  HARD  part in correcting the mistake is not even knowing where it is recorded  ,   let alone having access  .  ',
 " ' A great share of responsibility for this national tragedy unquestionably lies with the president of the country .   ' -- Eduard Shevardnadze  ,   former foreign minister ;   ' We are so deep in this crisis that all this business about leaving the party  ,   not leaving the party -- that will never get us out .   ' -- Natasha  ,   a Moscow bookke

In [91]:
SenSet

['HARD1', 'HARD2', 'HARD3']

In [101]:
##Use POS Tags if flag positive
# if POS_tags == 1:
#     for i in range(0, len(Sent)):
#         try:
#             tmp = word_tokenize(Sent[i])
#             tmp_tagged = pos_tag(tmp)
#         except UnicodeDecodeError as u:
#             Sent[i] = re.sub(r'[^\x00-\x7F]+',' ', Sent[i])
#             tmp = word_tokenize(Sent[i])
#             tmp_tagged = pos_tag(tmp)
#         tmp = ""
#         for j in range(0, len(tmp_tagged)):
#             tmptag = tmp_tagged[j][1].lower()

#             if POS_RED == 1:
#                 if tmptag in ['$', '\'', '(', ')', ',', '.', '--', ':', ';']:
#                     tmptag = 'PUNCTUATION'
#                 elif tmptag in ['DT', 'EX']:
#                     tmptag = 'DETERMINER'
#                 elif tmptag in ['jj', 'jjr', 'jjs']:
#                     tmptag = 'ADJ'
#                 elif tmptag in ['nn', 'nnp', 'nnps', 'nns']:
#                     tmptag = 'NOUN'
#                 elif tmptag in ['prp', 'prp$']:
#                     tmptag = 'PRONOUN'
#                 elif tmptag in ['rb', 'rbr','rbs']:
#                     tmptag = 'ADVERB'
#                 elif tmptag in ['vb', 'vbd', 'vbg','vbn','vbp','vbz']:
#                     tmptag = 'VERB'
#                 elif tmptag in ['wdt', 'wp', 'wp$', 'wrb']:
#                     tmptag = 'WH_WORD'

#             tmp += " " + tmp_tagged[j][0] + "_" + tmptag
#         Sent[i] = tmp

##Randomly shuffle sentences
tmp = list(zip(Sent, Sens))
random.shuffle(tmp)
Sent[:], Sens[:] = zip(*tmp)

In [102]:
Sent[0]

' He was the only skater to include two triple Axels in his long program and did the  HARDEST  combination of the night  ,   a triple Axel/triple Toe  .  '

In [94]:
##Get set of senses
lenst = len(Sent)
SenSet = list(set(Sens))
SenSet.sort()
SenCnt = []

##Get count of each sense
for i in SenSet:
    SenCnt.append(Sens.count(i))

##Most dominant sense
maxCnt = max(SenCnt)

Sent_div = []
for i in range(0,len(SenSet)):
    Sent_div.append([])

newline_regex = re.compile(r"\n[\n]*")
exclude = set(string.punctuation)


for i in range(0, lenst):
    Sent[i] = Sent[i].strip()

    ##remove stopwords if flag positive
    if remove_stopwords == 1:
        tmp = Sent[i].split()
        filtered_words = [word for word in tmp if word not in stopwords.words('english')]
        Sent[i] = ' '.join(filtered_words)

    ##remove punctuations if flag positive
    if remove_punctuation == 1:
        Sent[i] = ''.join(ch for ch in Sent[i] if ch not in exclude)

    ##extract window around the word if window positive
    if window == 1:
        left, _ ,right = Sent[i].lower().partition(checkword)
        n = windowSize
        left = left.split()[-n:]
        right = right.split()[:n]
        #print(right)
        #try:
        #    del right[0]
        #except:
        #    pass

        tmp = []
        if len(left) < windowSize and add_start_end == 1:
            for j in range(0,windowSize-len(left)):
                tmp.append('START')
        tmp.extend(left)
        left = tmp

        if add_start_end == 1:
            for j in range(len(right), windowSize):
                right.append('END')

        Sent[i] = ' '.join(left + [checkword] + right)

    ##Formatting the sentences
    Sent[i] = Sent[i].strip()
    Sent[i] = Sent[i].replace(" ", "\n")
    Sent[i] = newline_regex.sub("\n", Sent[i])
    if "\n\n" in Sent[i]:
        print(Sent[i])
    ind = SenSet.index(Sens[i])
    Sent_div[ind].append(Sent[i])

##Split Training and test sentences
SentTest = []
SensTest = []
Sent = []
Sens = []
for i in range(0, len(SenSet)):
    numTotal = len(Sent_div[i])
    numTest = int( split * numTotal)
    numTrain = numTotal - numTest

    SentTest.extend(Sent_div[i][ 0: numTest ])
    for j in range(0, numTest ):
        SensTest.append(SenSet[i])

    Sent.extend(Sent_div[i][ numTest: numTotal ])
    for j in range(0, numTrain ):
        Sens.append(SenSet[i])

##Equalize the number of Senses
lenst = len(Sent)
for i in range(0, lenst):
    ind = SenSet.index(Sens[i])
    num = int(1.0*maxCnt/SenCnt[ind])
    for j in range(0, num):
        Sent.append(Sent[i])
        Sens.append(Sens[i])

##Write training set sentences
tmp = list(zip(Sent, Sens))
random.shuffle(tmp)
Sent[:], Sens[:] = zip(*tmp)

print("Number of training examples: " + str(len(Sent)))
# fileSent = open("BLSTM/text_words.csv", "w")
# fileSens = open("BLSTM/summary_words.csv", "w")
# tmp = '\n\n'.join(Sent)
# fileSent.write(tmp)
# tmp = '\n\n'.join(Sens)
# fileSens.write(tmp)

# try:
#     ##Write test set sentences
#     tmp = list(zip(SentTest, SensTest))
#     random.shuffle(tmp)
#     SentTest[:], SensTest[:] = zip(*tmp)
#     print("Number of testing examples: " + str(len(SentTest)))

#     fileSentTest = open("BLSTM/test_text_words.csv", "w")
#     fileSensTest = open("BLSTM/test_summary_words.csv", "w")
#     tmp = '\n\n'.join(SentTest)
#     fileSentTest.write(tmp)
#     tmp = '\n\n'.join(SensTest)
#     fileSensTest.write(tmp)
# except:
#     print("Number of testing examples: 0")

Number of training examples: 4146


In [97]:
Sent[0]

"that\nit\n'\ns\nreally\nhard\nto\nget\na\nhandle\non"

In [96]:
Sens

['HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
 'HARD1',
